In [3]:
# CIEMSS ATE Query Example:

In [4]:
# begin
#     struct ATEQuery <: Query
#         output_var::Symbol
#         intervention_variable::Symbol
#         intervention_derivative_order::Integer
#         intervention_assignment::Real
#     end

#     function interpret(query::ATEQuery, model::Union{ODEModel, ODEProbabilisticModel})
#         twin_model = twinify(model)

#         intervention_variable = Symbol(string(query.intervention_variable) * "_cf")
#         intervention = ODEStaticIntervention(intervention_variable, query.intervention_derivative_order, query.intervention_assignment)
#         twin_model = intervene(twin_model, intervention)

#         output_cf = Symbol(string(query.output_var) * "_cf")
#         twin_model = add_variable(twin_model, :ATE, 0, :($(output_cf) - $(query.output_var)))
#     end
# end

In [ ]:
# AlgebraicJulia Workflow Wiring Diagram Example:

In [ ]:
# begin
#     # Form Workflow presentation of FreeBiproductCategory
#     @present Workflow(FreeBiproductCategory) begin
#         (File,LRN,TSpan,ODEProb,ODESol)::Ob 
#         MTKLoadLRN::Hom(File,LRN)
#         MTKFormODEProb::Hom(LRN⊗TSpan,ODEProb)
#         MTKSolveODE::Hom(ODEProb,ODESol)
#     end

#     # Form wiring diagram of load_form_sim Workflow
#     load_form_sim = @program Workflow (f::File,ts::TSpan) begin # 
#         lrn = MTKLoadLRN(f)
#         ode_prob = MTKFormODEProb(lrn,ts)
#         ode_sol = MTKSolveODE(ode_prob)
#         return ode_sol 
#     end

#     # Serialize program wiring diagram
#     # write_json_graph(load_form_sim,"diagram_load_form_sim.json") 
#     write_json_acset(load_form_sim.diagram, "diagram_load_form_sim.json")
# end

In [6]:
using Catlab, Catlab.Theories
using Catlab.CategoricalAlgebra
using Catlab.Graphics
using Catlab.Graphics: Graphviz
import Catlab.CategoricalAlgebra: migrate!
using Catlab.WiringDiagrams
using Catlab.Programs
using Catlab.Programs.RelationalPrograms

In [16]:
# Form Workflow presentation of FreeBiproductCategory
@present Workflow(FreeBiproductCategory) begin
    
    (Output_Var, Intervention_Variable, Intervention_Derivative_Order, Intervention_Assignment, Query, Model, Twin_Model, Intervention, Twin_Model_Int, Comparison)::Ob
    
    ConstructQuery::Hom(Output_Var⊗Intervention_Variable⊗Intervention_Derivative_Order⊗Intervention_Assignment, Query)
    
    Twinify::Hom(Model, Twin_Model)
    
    ODEStaticIntervention::Hom(Intervention_Variable⊗Query, Intervention)
    
    Intervene::Hom(Twin_Model⊗Intervention, Twin_Model_Int)
    
    Compare::Hom(Query⊗Twin_Model⊗Twin_Model_Int, Comparison)
    
end

Presentation{ThBiproductCategory, Symbol}(Catlab.Theories.FreeBiproductCategory, (Ob = Catlab.Theories.FreeBiproductCategory.Ob{:generator}[Output_Var, Intervention_Variable, Intervention_Derivative_Order, Intervention_Assignment, Query, Model, Twin_Model, Intervention, Twin_Model_Int, Comparison], Hom = Catlab.Theories.FreeBiproductCategory.Hom{:generator}[ConstructQuery, Twinify, ODEStaticIntervention, Intervene, Compare]), Dict(:Intervention_Assignment => (:Ob => 4), :Intervention => (:Ob => 8), :Output_Var => (:Ob => 1), :Twin_Model => (:Ob => 7), :ODEStaticIntervention => (:Hom => 3), :Comparison => (:Ob => 10), :Query => (:Ob => 5), :Twinify => (:Hom => 2), :Intervention_Variable => (:Ob => 2), :Model => (:Ob => 6)…), Pair[])

In [17]:
# Form wiring diagram of Workflow
ate_workflow = @program Workflow (output_var::Output_Var, intervention_variable::Intervention_Variable, intervention_derivative_order::Intervention_Derivative_Order, intervention_assignment::Intervention_Assignment, model::Model) begin
    
    query = ConstructQuery(output_var, intervention_variable, intervention_derivative_order, intervention_assignment)
    
    twin_model = Twinify(model)
    
    intervention = ODEStaticIntervention(intervention_variable, query)
    
    twin_model_int = Intervene(twin_model, intervention)
    
    comparison = Compare(query, twin_model, twin_model_int)
    
    return comparison
end

WiringDiagram{ThBiproductCategory}([:Output_Var,:Intervention_Variable,:Intervention_Derivative_Order,:Intervention_Assignment,:Model], [:Comparison], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box(:ConstructQuery, [:Output_Var,:Intervention_Variable,:Intervention_Derivative_Order,:Intervention_Assignment], [:Query]),
  2 => Box(:Twinify, [:Model], [:Twin_Model]),
  3 => Box(:ODEStaticIntervention, [:Intervention_Variable,:Query], [:Intervention]),
  4 => Box(:Intervene, [:Twin_Model,:Intervention], [:Twin_Model_Int]),
  5 => Box(:Compare, [:Query,:Twin_Model,:Twin_Model_Int], [:Comparison]) ],
[ Wire((-2,1) => (1,1)),
  Wire((-2,2) => (1,2)),
  Wire((-2,3) => (1,3)),
  Wire((-2,4) => (1,4)),
  Wire((-2,5) => (2,1)),
  Wire((-2,2) => (3,1)),
  Wire((2,1) => (5,2)),
  Wire((1,1) => (5,1)),
  Wire((2,1) => (4,1)),
  Wire((1,1) => (3,2)),
  Wire((3,1) => (4,2)),
  Wire((4,1) => (5,3)),
  Wire((5,1) => (-1,1)) ])

In [18]:
# Serialize program wiring diagram
write_json_acset(ate_workflow.diagram, "ATE_dwd.json")

1989